In [1]:
from pathlib import Path


# classes_path = Path('../input/train_simplified/')
cls_path = Path('../input/train_simplified/')
# iter_cls_path = cls_path.iterdir()

class_dict = {x.stem.replace(" ", "_"):i for i, x in enumerate(cls_path.iterdir())}
labels = {x.stem.replace(" ", "_") for i, x in enumerate(cls_path.iterdir())}

n_labels = len(labels)
print("Number of labels: {}".format(n_labels))

FileNotFoundError: [Errno 2] No such file or directory: '../input/train_simplified'

In [5]:
import numpy as np
import cv2


BASE_SIZE=256


def drawing_to_np(raw_strokes, size=64, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                        (stroke[0][i+1], stroke[0][i+1]), color, lw)
    
    return cv2.resize(img, (size, size)) if size != BASE_SIZE else img
    

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import pdb

max_img_per_class = 10
wshape = 64
hshape = 64
# to image from stroke
def drawing_to_np(drawing, shape=(64, 64)):
    drawing = ast.literal_eval(drawing)
    fig, ax = plt.subplots()
    for x,y in drawing:
        ax.plot(x, y, marker='.')
        ax.axis('off')
    
    fig.canvas.draw()
    # Convert images to numpy arrat
    np_drawing = np.array(fig.canvas.renderer._renderer)
    plt.close(fig)
#     pdb.set_trace()
    img = cv2.resize(np_drawing, shape)
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img_expanded = img_gray[:, :, np.newaxis]
#     img_expanded = img_gray[np.newaxis, :, :]
#     pdb.set_trace()
    return img_gray
#     return img_expanded
#     return img


    

def path_to_img(path):
    train = pd.read_csv(path, usecols=['word', 'drawing'], nrows=max_img_per_class)
    train['word'] = train.word.str.replace(" ", "_")
    train['drawing_np'] = train['drawing'].map(drawing_to_np)
    label = train.word.map(class_dict)
    return  train['drawing_np'], label



In [6]:
from PIL import Image
import ast

def save_image(image_numpy, image_path):
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(image_path)

def csv2img(fn, save_path=Path('.')):
    n_row = 100
    train = pd.read_csv(fn, usecols=['word', 'drawing'], nrows=n_row)
    train['word'] = train.word.str.replace(" ", "_")
    train['drawing'] = train['drawing'].apply(ast.literal_eval)
    train['drawing_np'] = train['drawing'].apply(drawing_to_np, True, shape=(128, 128))
#     label = train.word.map(class_dict)
#     return  train['drawing_np'], label
    for i, np_drawing in enumerate(train['drawing_np']):
        gray_img = np_drawing[..., np.newaxis]
        save_image(gray_img, save_path/f'car{i}.png')
        
    